In [1]:
import pandas as pd

## 데이터

In [3]:
origin_data = pd.read_csv('./final_vod.csv', encoding = 'cp949')

In [ ]:
origin_data.head(1)

In [7]:
genre_data = origin_data[['subsr', 'ct_cl']]

In [8]:
genre_data.head(2)

,subsr,ct_cl
0,65941000,TV 시사/교양
1,66873000,TV 시사/교양


In [ ]:
genre_data.groupby('subsr').count().reset_index()

## 시청 데이터가 많은 시청자들 중 다양한 장르를 시청한 시청자 탐색

In [ ]:
subsr_list = ['']

for subsr in subsr_list:
    chr = genre_data[genre_data['subsr'] == subsr]
    if chr['ct_cl'].nunique() > 3:
        print('subsr :', subsr)
        #chr['ct_cl'].nunique()
        print(chr.groupby('ct_cl').count())

In [28]:
chr = genre_data[genre_data['subsr'] == 64154000]
chr.head()

,subsr,ct_cl
24,64154000,TV 시사/교양
103,64154000,TV 시사/교양
441,64154000,TV 연예/오락
938,64154000,TV 연예/오락
1231,64154000,TV드라마


In [31]:
chr['ct_cl'].nunique()

4

In [ ]:
# 가장 시청이 많은 1명과 다양하게 시청한 1명을 가지고 테스트
main_subsr = [64154000, 64659000]

## 선정한 2명을 가지고 추천 수행

### 시청 시간 기반 추천

In [33]:
run_data = origin_data[['use_tms', 'disp_rtm_s', 'asset_nm_new']]

In [51]:
# 러닝 타임 기반 추천(사람과 상관 없이 공통 추천)
def runningtime_recommendation(data):
    # 시청 기록을 저장할 dict
    watch_dict = {}
    
    # 데이터프레임을 순회하면서 러닝 타임의 합계와 시청 횟수 계산
    for _, item in data.iterrows():
        running_time = item['disp_rtm_s']
        watch_time = item['use_tms']
        content_name = item['asset_nm_new']

        # 아직 dict 가 생성되지 않은 경우
        if content_name not in watch_dict:
            watch_dict[content_name] = [watch_time, running_time, 1]
        # 이미 dict 에 데이터가 들어있는 경우
        else:
            value = watch_dict[content_name]
            watch_dict[content_name] = [value[0] + watch_time, value[1] + running_time, value[2] + 1]
    
    # 러닝타임 대비 시청 시간 비율 계산
    for item in watch_dict:
        w, r, times = watch_dict[item]
        if r != 0.0:
            watch_dict[item] = [w / r, times]
        else:
            watch_dict[item] = [0, times]
            
    # 시청 횟수 기준 설정
    watch_times_limit = 5
    del_times_list = []
    
    # 시청 횟수 기준 이하인 리스트 생성
    for item in watch_dict:
        if watch_dict[item][1] <= watch_times_limit:
            del_times_list.append(item)
    # 시청 횟수가 기준 이하인 데이터 삭제        
    for item in del_times_list:
        del(watch_dict[item])
        
            
    # 러닝 타임 대비 시청 시간 비율 순서대로 정렬
    sorted_dict = sorted(watch_dict.items(), key = lambda item: item[1], reverse = True)
    
    # 러닝 타임 대비 시청 시간 비율이 높은 순서대로 상위 10개 출력
    for item in sorted_dict[:10]:
        print(item[0])

In [ ]:
runningtime_recommendation(run_data)

In [55]:
non_kids = origin_data[origin_data['ct_cl'] != '키즈']

In [ ]:
non_kids_run_data = non_kids[['use_tms', 'disp_rtm_s', 'asset_nm_new']]

runningtime_recommendation(non_kids_run_data)

### 줄거리 기반 추천

In [71]:
from konlpy.tag import Twitter
import math
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [69]:
open_file = open('./text files/detail_genre.txt', 'r', encoding = 'utf8')
text = open_file.read()
open_file.close()

# 파일에서 읽어온 데이터 확인
#print(text)

data_list = text.split('\n')

# data_list 의 마지막에 공백이 추가되므로 공백 제거
data_list = data_list[:-2]
# 확인
#print(data_list)

In [72]:
with open('./text files/stopword.txt', 'r', encoding = 'utf-8') as f:
    stopwords = f.read()

#print(type(stopwords))
stopwords_list = stopwords.split('\n')

word_dict_data = {}

# 각 세부 장르 별로 단어 리스트 생성하고 저장
for detail_genre in data_list:
    # 각 세부 장르에 대해 단어 데이터 가져오기
    open_file = open('./text files/' + detail_genre + '_words.txt', 'r', encoding = 'utf8')
    text = open_file.read()
    
    # 파일 닫기
    open_file.close()
    
    # 각 단어 분리하기
    text_list = text.split('\n')
    text_list = text_list[:-2] # 마지막의 공백 제외
    
    # 각 세부 장르별 줄거리의 단어 갯수 확인 => 가장 단어의 갯수가 적은 경우는 15개
    #no_dup = set(text)
    #print(detail_genre, ' : ', len(text), len(no_dup))
    
    # 각 장르별 주요 단어 찾기
    spliter = Twitter()
    nouns_for_dictionary = spliter.nouns(text)
    ko = nltk.Text(nouns_for_dictionary, name = detail_genre)
    
    
    # 불용어 제거
    ko1 = [each_word for each_word in ko if each_word not in stopwords_list]
    ko = nltk.Text(ko1, name = detail_genre)
    word_dict = dict(ko.vocab())

    # 확인
    #print(type(word_dict))
    sorted_dictionary = sorted(word_dict.items(), key = lambda item: item[1], reverse = True)
    #print(sorted_dictionary)
    
    # 단어 리스트를 dict 형태로 저장
    word_dict_data[detail_genre] = word_dict

In [79]:
# 장르별 벡터 저장
word_vector_dict = {}

vectorizer = CountVectorizer(min_df = 0.05)
    
# dict 형태로 저장되어 있는 단어 데이터를 읽고
# 각 단어들을 모아서 하나의 문장으로 만든 다음 저장
for detail_genre in data_list:
    contents_tokens = word_dict_data[detail_genre]

    # 벡터화를 위해 단어들을 가지고 문장 생성
    contents_for_vect = []
    sentence = ''
    # 토큰 단위로 구분된 문장을 생성
    for content in contents_tokens:
        sentence += ' ' + content

    # 생성한 문장을 리스트에 추가
    contents_for_vect.append(sentence)
    
    word_vector_dict[detail_genre] = contents_for_vect

In [82]:
#거리 구해주는 함수 생성
def dist_raw(v1, v2):
    # 차이를 계산
    delta = v1 - v2
    
    return sp.linalg.norm(delta.toarray())

In [84]:
# 사용자 번호를 입력으로 사용
def genre_rec(subsr_num):
    # 사용자 번호가 일치하는 데이터 중 줄거리만 가져오기
    user_data = origin_data[origin_data['subsr'] == subsr_num]['SMRY']
    
    smry_list = []
    # 데이터를 순회하면서 줄거리 저장
    for item in user_data:
        if item not in smry_list:
            smry_list.append(item)
        
    smry_sentence = ''
    for smry in smry_list:
        smry_sentence += (smry + ' ')
        
    # 샘플 문장 토큰화
    spliter = Twitter()
    sample_words = spliter.nouns(smry_sentence)

    # 가장 거리가 짧은 세부 장르 계산용
    min_distance = 65536
    min_detail_genre = 'None'

        
    for detail_genre in data_list:
        vectorizer = CountVectorizer(min_df = 1) # 1번만 등장해도 단어 사전에 포함시키도록

        # 장르 별로 줄거리 불러오기 - dict 의 value
        contents_tokens = word_vector_dict[detail_genre]

        sentence = contents_tokens[0]

        # 생성한 문장을 리스트에 추가
        contents_for_vect = []
        contents_for_vect.append(sentence)

        # 피처 벡터화 - 띄어쓰기를 기준으로 벡터화
        X = vectorizer.fit_transform(contents_for_vect)

        # 샘플 줄거리 문장을 피처 벡터화
        new_content_vect = vectorizer.transform([smry_sentence])

        # 거리 계산
        post_vec = X.getrow(0) # 단어 문장이 1개이므로 첫번째인 0번 인덱스
        distance = dist_raw(new_content_vect, post_vec)

        # 세부 장르별 단어의 갯수를 세고 일정 갯수 이하이면 제외
        length = post_vec.shape[1]
        limit_length = 100
        limit_cor = 0.3
        if length < limit_length:
            pass

        else:
            # 일치율 계산하기
            count = 0
            for item in sample_words:
                if item in contents_tokens[0]:
                    count += 1

            cor = count / len(sample_words)

            # 일치율이 기준을 넘는 경우에만 거리를 계산
            if cor < limit_cor:
                pass

            else:
                # 결과 확인 - 장르와의 거리는 줄거리 단어의 길이를 가중치로 반영하여 계산
                weighted_distance = distance / (math.log10(length)) / cor
                print(detail_genre, ' 장르와의 거리는 : \t', weighted_distance, sep='') 
                # 1000을 곱한 이유는 쉽게 보기 위함
                print(detail_genre, ' 장르와의 거리(원본)는 : \t', distance, sep='')
                print(detail_genre, ' 장르와의 일치율은 : \t', cor * 100 , '%', sep='')
                print(detail_genre, ' 장르의 단어 길이는 : \t', length, '\n',sep='')

                if weighted_distance < min_distance:
                    min_distance = weighted_distance
                    min_detail_genre = detail_genre

    # 최종 결과         
    print('가장 유사한 장르는 ', min_detail_genre, sep = '')

In [ ]:
genre_rec(64154000)

In [ ]:
genre_rec(64659000)

### 출연진 기반 추천

In [91]:
drop_data = origin_data[['asset_nm_new', 'ACTR_DISP']]

In [127]:
from collections import Counter

# 사용자 번호를 입력받아서 시청한 컨텐츠에 출연한 출연진 기반 추천
def cast_recommendaton(subsr_num):
    view_data = origin_data[origin_data['subsr'] == subsr_num][['asset_nm_new','ACTR_DISP']]
    
    # 출연진 정보를 저장할 list
    actr_list = []

    # 시청 데이터를 순회하면서 출연진 정보를 저장
    for _, item in view_data.iterrows():
        actr = item['ACTR_DISP']
        
        if ',' in actr:
            item_list = actr.split(',')
        else:
            item_list = [actr]

        for actr in item_list:
            if actr != '-':
                actr_list.append(actr)
            
    count = Counter(actr_list)

    # 가장 많이 출연한 상위 N 명의 출연진 정보
    N_people = 30
    bestN_actr = count.most_common(N_people)
    print(bestN_actr)
    
    # 상위 N위 출연진에 대해 가중치 설정
    score_sum = 0
    for actr_score in bestN_actr:
        score_sum += actr_score[1]

    w_bestN = []
    # 출연진 가중치 점수를 소수 아래 2자리 반올림
    for actr_score in bestN_actr:
        w_bestN.append([actr_score[0], round((actr_score[1] / score_sum), 2)])
        
        
    actr_score_list = []
    # 각 행을 순회하면서 출연진 정보에 대한 점수를 더해서 데이터 생성
    for _, item in drop_data.iterrows():
        score_data = 0

        for name_score in w_bestN:
            if name_score[0] in str(item['ACTR_DISP']):
                score_data += name_score[1]
        actr_score_list.append(score_data)

    # 기존 데이터에 컬럼 추가
    drop_data['ATCR_SCORE'] = actr_score_list

    # 출연진 점수를 기준으로 순위 부여
    drop_data['rank'] = drop_data['ATCR_SCORE'].rank(method = 'min', na_option = 'bottom',
                                                    ascending = False)
    
    count_data = drop_data.groupby(['asset_nm_new', 'ATCR_SCORE']).count().reset_index()
    
    # ATCR_SCORE 를 기준으로 정렬을 수행
    sorted_ac_data = count_data.sort_values(by = ['ATCR_SCORE', 'asset_nm_new'], ascending = False)
    print(sorted_ac_data)
    
    # 상위 10개 컨텐츠의 정보를 리스트로 생성
    top10 = list(sorted_ac_data['asset_nm_new'][:10])
    print(top10)

In [ ]:
cast_recommendaton(64154000)

In [ ]:
cast_recommendaton(64659000)

## 추천

In [ ]:
# 시청 시간 비율 기반 추천
runningtime_recommendation(run_data)

In [ ]:
# 시청 시간 비율 기반 추천 - 키즈 장르 제외
runningtime_recommendation(non_kids_run_data)

In [ ]:
# 줄거리 기반 장르 추천
genre_rec(64154000)

In [ ]:
# 줄거리 기반 장르 추천
genre_rec(64659000)

In [ ]:
# 출연진 기반 추천
cast_recommendaton(64154000)

In [ ]:
# 출연진 기반 추천
cast_recommendaton(64659000)